In [1]:
using Revise
include("ngrams.jl")
include("datasetloader.jl")
LANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", 
"eng", "epo", "fas", "fin", "fra", "hau", "hbs", "heb", "hin", "hun", 
"ido", "ina", "isl", "ita", "jpn", "kab", "kor", "kur", "lat", "lit", 
"mar", "mkd", "msa", "nds", "nld", "nor", "pol", "por", "ron", "rus", 
"slk", "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", 
"zho"]
G = [lang => load_ngrams(joinpath("ngrams", lang * ".txt")) for lang in LANGS];
sort([l=>sum(values(D)) for (l,D) in G], by=last, rev=true)


50-element Vector{Pair{String, Float64}}:
 "rus" => 32.13508769119433
 "mkd" => 30.258449399847375
 "mar" => 29.977432692509925
 "ukr" => 29.55481747742666
 "ben" => 29.503371192249226
 "hin" => 28.912644494697663
 "heb" => 28.874006712817305
 "bul" => 28.71517938937598
 "hbs" => 27.68471530085054
 "yid" => 27.473338723106526
       ⋮
 "vie" => 22.190633137196794
 "isl" => 22.104003948087577
 "lit" => 21.812230613518143
 "ces" => 21.74667931504059
 "slk" => 21.731447490477912
 "jpn" => 21.461441564113343
 "pol" => 21.308682471129547
 "kor" => 18.807540715647907
 "zho" => 16.602311389017885

In [2]:
function norm_table!(t)
    D = last(t)
    vs = sum(values(D))
    for (k, v) in D
        D[k] /= vs
        D[k] = log(D[k])
    end
end
function likelihood(t, logt, default_logp=DEFAULT_LOGP)
    sc = 0.
    for (code, p) in t
        sc += p * get(logt, code, default_logp)
    end
    sc
end
function detector_lh(text; ngram=5, languages=LANGS, logtable=LOG_T)
    t = merged_ngrams(text, ngram)
    lhs = likelihood.(Ref(t), logtable)
    languages[argmax(lhs)]
end

detector_lh (generic function with 1 method)

In [3]:
norm_table!.(G);
LOG_T = last.(G);
DEFAULT_LOGP = (minimum.(values.(last.((G))))|>minimum)


-16.96272550019555

In [4]:
include("benchmark.jl")
WV = WikiDataSet("corpus/wikipedia/test", langs=LANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [5]:
WB = benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh(t, ngram=4),
    "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=LANGS)

likelihood-n4

127.426978 seconds (1.63 G allocations: 28.846 GiB, 4.42% gc time, 0.59% compilation time)


likelihood-n3

 76.725221 seconds (885.37 M allocations: 16.355 GiB, 4.33% gc time, 0.20% compilation time)


2-element Vector{Any}:
 "likelihood-n4" => [1.0, 0.99, 1.0, 1.0, 1.0, 0.985, 0.995, 0.965, 1.0, 1.0  …  1.0, 0.99, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
 "likelihood-n3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.985, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.98, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]

In [6]:
showtable(WB, LANGS, threshold=-0.5)

|               | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likeliho

|               | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n4 |  97.63% | 100.00% | 99.00% | 100.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% | 66.50% | 100.00% | 90.00% |  99.50% | 93.00% | 93.00% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 95.00% | 98.00% | 99.00% | 99.00% | 97.50% | 93.50% |  99.50% | 99.50% | 91.00% | 100.00% | 96.50% | 100.00% | 99.00% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% | 99.50% |
| likelihood-n3 |  95.93% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.00% | 10.50% | 100.00% | 89.50% | 100.00% | 88.50% | 84.50% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.00% | 97.00% | 99.00% | 98.50% | 96.50% | 91.00% | 100.00% | 99.50% | 90.00% |  99.50% | 96.50% | 100.00% | 98.50% | 98.00% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% | 99.50% |


In [7]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=("mkd", "hbs"))

likelihood-n5

  9.023298 seconds (90.31 M allocations: 1.666 GiB, 3.74% gc time, 1.14% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.99, 0.88]

In [8]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=TV, languages=("mkd", "hbs"))

likelihood-n5

 32.151567 seconds (117.73 M allocations: 2.000 GiB, 1.17% gc time, 0.05% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.9469910371318822, 0.21888053467000834]